In [1]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [40]:
# 1. Function to calculate r_g, random_entropy, and waiting_times
def calculate_metrics(df_chunk):
    # Calculate centroid and r_g
    centroid_lon = df_chunk['lon'].mean()
    centroid_lat = df_chunk['lat'].mean()
    distances = np.sqrt((df_chunk['lon'] - centroid_lon)**2 + (df_chunk['lat'] - centroid_lat)**2)
    rg = np.sqrt((distances**2).mean())
    
    # Calculate random entropy (based on the distribution of locations)
    position_counts = df_chunk.groupby(['lon', 'lat']).size()
    probabilities = position_counts / position_counts.sum()
    random_entropy = -np.sum(probabilities * np.log2(probabilities))
    
    # Calculate waiting times
    df_chunk = df_chunk.sort_values(by='timestamp')
    df_chunk['time_diff'] = df_chunk['timestamp'].diff().dt.total_seconds().fillna(0)
    waiting_times = df_chunk.groupby(['lon', 'lat'])['time_diff'].sum().mean()
    
    return pd.Series({
        'rg': rg,
        'random_entropy': random_entropy,
        'waiting_times': waiting_times
    })

# 2. Function to fetch data from the database and calculate r_g based on different time periods
def fetch_data_by_time_period(time_period):
    # Construct the query based on the time period
    if time_period == 'week':
        date_trunc = "DATE_TRUNC('week', gps_logs.logtime)"
    elif time_period == '2weeks':
        date_trunc = "FLOOR(EXTRACT('epoch' FROM gps_logs.logtime) / (2 * 7 * 24 * 60 * 60)) * (2 * 7 * 24 * 60 * 60)"
    elif time_period == '3weeks':
        date_trunc = "FLOOR(EXTRACT('epoch' FROM gps_logs.logtime) / (3 * 7 * 24 * 60 * 60)) * (3 * 7 * 24 * 60 * 60)"
    elif time_period == 'month':
        date_trunc = "DATE_TRUNC('month', gps_logs.logtime)"
    else:
        raise ValueError("Unsupported time period")
    
    query = f"""
    WITH centroid_per_user AS (
        SELECT 
            uuid, 
            ST_Centroid(ST_Collect(geom)) AS centroid_geom,
            {date_trunc} AS time_period
        FROM blogwatcher.gps_logs gps_logs
        WHERE gps_logs.logtime BETWEEN '2023-07-22' AND '2023-08-22'
        GROUP BY uuid, time_period
    )
    SELECT 
        gps_logs.uuid, 
        {date_trunc} AS time_period,
        SQRT(SUM(POW(ST_Distance(gps_logs.geom, c.centroid_geom), 2)) / COUNT(*)) AS radius_of_gyration
    FROM 
        blogwatcher.gps_logs gps_logs
    JOIN 
        centroid_per_user c 
    ON 
        gps_logs.uuid = c.uuid AND {date_trunc} = c.time_period
    WHERE 
        gps_logs.logtime BETWEEN '2023-07-22' AND '2023-08-22'
    GROUP BY 
        gps_logs.uuid, time_period
    ORDER BY 
        time_period;
    """
    
    # Execute the query and return the results
    conn = psycopg2.connect(
        dbname = "gisdb", # "bw_tokyo/bw_ishikawa" 
        user = "postgres",
        password =  "task4TH",
        host = "localhost",
        port = '5432', 
    )
    df = pd.read_sql(query, conn)
    conn.close()
    
    return df

# 3. Visualization function
def compare_and_visualize(df_metrics_gps, df_metrics_pflow):
    plt.figure(figsize=(18, 8))
    
    # 1. Comparison of r_g probability distribution
    plt.subplot(1, 3, 1)
    for label, df in zip(['GPS', 'Pseudo PFLOW'], [df_metrics_gps, df_metrics_pflow]):
        rg_values = df['rg'].dropna().values
        hist, bin_edges = np.histogram(rg_values, bins=np.logspace(np.log10(1), np.log10(1000), 50), density=True)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        plt.loglog(bin_centers, hist, 'o-', label=label)
    plt.xlabel(r'$r_g$ (km)')
    plt.ylabel(r'$P(r_g)$')
    plt.title('Comparison of r_g Probability Distribution')
    plt.legend()
    plt.grid(True, which="both", ls="--")
    
    # 2. Comparison of random_entropy density plot
    plt.subplot(1, 3, 2)
    for label, df in zip(['GPS', 'Pseudo PFLOW'], [df_metrics_gps, df_metrics_pflow]):
        df['random_entropy'].plot(kind='density', label=label)
    plt.xlabel('Entropy')
    plt.ylabel('Density')
    plt.title('Comparison of Random Entropy Density Plot')
    plt.legend()
    plt.grid(True)
    
    # 3. Comparison of waiting_times histogram
    plt.subplot(1, 3, 3)
    for label, df in zip(['GPS', 'Pseudo PFLOW'], [df_metrics_gps, df_metrics_pflow]):
        plt.hist(df['waiting_times'].dropna().values, bins=50, alpha=0.7, label=label)
    plt.xlabel('Waiting Time (seconds)')
    plt.ylabel('Frequency')
    plt.title('Comparison of Waiting Times Histogram')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

In [41]:
# Load and analyze GPS data for different time periods
df_metrics_gps_week = fetch_data_by_time_period('week')
df_metrics_gps_week

/tmp/ipykernel_3538583/3809587830.py:75: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


DatabaseError: Execution failed on sql '
    WITH centroid_per_user AS (
        SELECT 
            uuid, 
            ST_Centroid(ST_Collect(geom)) AS centroid_geom,
            DATE_TRUNC('week', gps_logs.logtime) AS time_period
        FROM blogwatcher.gps_logs gps_logs
        WHERE gps_logs.logtime BETWEEN '2023-07-22' AND '2023-08-22'
        GROUP BY uuid, time_period
    )
    SELECT 
        gps_logs.uuid, 
        DATE_TRUNC('week', gps_logs.logtime) AS time_period,
        SQRT(SUM(POW(ST_Distance(gps_logs.geom, c.centroid_geom), 2)) / COUNT(*)) AS radius_of_gyration
    FROM 
        blogwatcher.gps_logs gps_logs
    JOIN 
        centroid_per_user c 
    ON 
        gps_logs.uuid = c.uuid AND DATE_TRUNC('week', gps_logs.logtime) = c.time_period
    WHERE 
        gps_logs.logtime BETWEEN '2023-07-22' AND '2023-08-22'
    GROUP BY 
        gps_logs.uuid, time_period
    ORDER BY 
        time_period;
    ': column "gps_logs.logtime" must appear in the GROUP BY clause or be

In [ ]:
df_metrics_gps_2weeks = fetch_data_by_time_period('2weeks')
df_metrics_gps_2weeks

In [ ]:
df_metrics_gps_3weeks = fetch_data_by_time_period('3weeks')
df_metrics_gps_3weeks

In [ ]:
df_metrics_gps_month = fetch_data_by_time_period('month')
df_metrics_gps_month

In [ ]:
# Load and analyze Pseudo PFLOW data
df_metrics_pflow = analyze_pseudo_pflow('path_to_pseudo_pflow.csv')
df_metrics_pflow

In [ ]:
# Compare and visualize the results
compare_and_visualize(df_metrics_gps_week, df_metrics_pflow)